In [1]:
import os
os.getcwd()

'/root/RAG'

In [2]:
# === RAG: 基础配置 ===
import os, json, math, pickle, gc
from pathlib import Path
from typing import List, Dict

import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
import faiss
from tqdm import tqdm

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

import sys
sys.path.append("/root/customs_tokenizers/")  

BASE_MODEL_DIR    = "/root/bert-base-chinese"                       # 本地 BERT
LORA_WEIGHT       = "/root/models/model_best_stroke_lora.pth"       # LoRA 权重
MERGED_ENCODER    = "/root/models/model_merged_stroke"             
QWEN_DIR          = "/root/autodl-tmp/Qwen2.5-1.5B"                 # 本地 Qwen 1.5B
CORPUS_JSON       = "RAG_data_50k.json"                      # 语料
RAG_DIR           = Path("/root/RAG")
RAG_DIR.mkdir(parents=True, exist_ok=True)

INDEX_PATH        = RAG_DIR / "faiss.index"
CHUNKS_META_PATH  = RAG_DIR / "chunks.pkl"
from pinyin_tokenizer import PinyinTokenizer
from stroke_tokenizer import StrokeTokenizer

# RAG 参数
CHUNK_SIZE   = 250    
CHUNK_OVERLAP = 40    
TOP_K        = 5    
BATCH_SIZE   = 64      
MAX_LEN      = 128    

/root/miniconda3/lib/python3.8/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/root/miniconda3/lib/python3.8/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issu

In [3]:
from peft import LoraConfig, get_peft_model, PeftModel

def load_encoder():
    # 优先加载合并后的编码器
    if Path(MERGED_ENCODER).exists():
        tok = AutoTokenizer.from_pretrained(MERGED_ENCODER, local_files_only=True)
        enc = AutoModel.from_pretrained(MERGED_ENCODER, local_files_only=True).to(DEVICE).eval()
        print("Encoder loaded (merged). Hidden:", enc.config.hidden_size)
        return tok, enc

    # 否则按 LoRA 方式加载
    tok = AutoTokenizer.from_pretrained(BASE_MODEL_DIR, local_files_only=True)
    base = AutoModel.from_pretrained(BASE_MODEL_DIR, local_files_only=True)

    # 与训练一致的 LoRA 配置
    peft_cfg = LoraConfig(
        r=8, lora_alpha=16, target_modules=["query","key","value"],
        lora_dropout=0.1, bias="none"
    )
    enc = get_peft_model(base, peft_cfg)

    # 加载权重（修正键名前缀）
    sd = torch.load(LORA_WEIGHT, map_location="cpu")
    new_sd = { (k.replace("base_model.model.","base_model.") if k.startswith("base_model.model.") else k): v
               for k,v in sd.items() }
    try:
        enc.load_state_dict(new_sd, strict=True)
    except Exception:
        enc.load_state_dict(new_sd, strict=False)

    enc = enc.to(DEVICE).eval()
    print("Encoder loaded (LoRA). Hidden:", enc.base_model.config.hidden_size if hasattr(enc,"base_model") else enc.config.hidden_size)
    return tok, enc

enc_tok, enc_model = load_encoder()

Encoder loaded (LoRA). Hidden: 768


In [4]:
def load_llm():
    llm_tok = AutoTokenizer.from_pretrained(QWEN_DIR, trust_remote_code=True, local_files_only=True)
    llm = AutoModelForCausalLM.from_pretrained(
        QWEN_DIR, trust_remote_code=True, local_files_only=True
    ).to(DEVICE).eval()
    return llm_tok, llm

llm_tok, llm = load_llm()
print("LLM ready.")

LLM ready.


In [5]:
def load_corpus(json_path: str) -> List[Dict]:
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    # 兼容 [{'query','document'}] 或 [{'question','answer'}]
    out = []
    for r in data:
        doc = r.get("document") or r.get("answer") or ""
        q   = r.get("query") or r.get("question") or ""
        if doc:
            out.append({"query": q, "document": doc})
    return out

from stroke_tokenizer import StrokeTokenizer
tok_stroke = StrokeTokenizer()

from pinyin_tokenizer import PinyinTokenizer

tok_pinyin = PinyinTokenizer()

def chunk_text(text: str, size=CHUNK_SIZE, overlap=CHUNK_OVERLAP) -> List[str]:
    """基于拼音 tokenizer 的 token 级分块"""
    token_ids = tok_pinyin.encode(text)
    chunks = []
    start = 0
    while start < len(token_ids):
        end = start + size
        sub_ids = token_ids[start:end]
        # 把 token id 转回对应 token
        sub_tokens = [tok_pinyin.id2token[i] for i in sub_ids if i in tok_pinyin.id2token]
        chunk = "".join(sub_tokens)
        chunks.append(chunk)
        if end >= len(token_ids):
            break
        start = end - overlap
    return chunks

corpus = load_corpus(CORPUS_JSON)
print("Loaded docs:", len(corpus))

# 构建 chunk 元信息
chunks = []
for i, r in enumerate(corpus):
    for c in chunk_text(r["document"], CHUNK_SIZE, CHUNK_OVERLAP):
        chunks.append({"doc_id": i, "text": c})
print("Total chunks:", len(chunks))

Loaded docs: 45000
Total chunks: 115214


In [6]:
@torch.inference_mode()
def encode_texts(texts: List[str]) -> torch.Tensor:
    all_vecs = []
    for i in tqdm(range(0, len(texts), BATCH_SIZE), desc="Encoding"):
        batch = texts[i:i+BATCH_SIZE]
        enc = enc_tok(batch, return_tensors="pt", padding=True, truncation=True, max_length=MAX_LEN)
        enc = {k: v.to(DEVICE) for k, v in enc.items()}
        out = enc_model(**enc, output_hidden_states=True)
        cls = out.hidden_states[-1][:, 0, :]  # [CLS]
        all_vecs.append(cls.detach().float().cpu())
    return torch.cat(all_vecs, dim=0) if all_vecs else torch.empty(0, dtype=torch.float32)

def build_or_load_index(chunks, index_path=INDEX_PATH, meta_path=CHUNKS_META_PATH):
    if index_path.exists() and meta_path.exists():
        index = faiss.read_index(str(index_path))
        with open(meta_path, "rb") as f:
            meta = pickle.load(f)
        print("Index loaded:", index.ntotal)
        return index, meta

    texts = [c["text"] for c in chunks]
    vecs = encode_texts(texts).numpy().astype("float32")  # [N, D]
    dim = vecs.shape[1]
    index = faiss.IndexFlatIP(dim)
    # 向量先归一化，可用余弦相似度
    faiss.normalize_L2(vecs)
    index.add(vecs)

    faiss.write_index(index, str(index_path))
    with open(meta_path, "wb") as f:
        pickle.dump(chunks, f)
    print("Index built:", index.ntotal, "dim:", dim)
    return index, chunks

index, chunks_meta = build_or_load_index(chunks)

Index loaded: 11001


In [7]:
def retrieve(query: str, top_k=TOP_K):
    qv = encode_texts([query]).numpy().astype("float32")
    faiss.normalize_L2(qv)
    D, I = index.search(qv, top_k)   # 余弦相似度
    I = I[0].tolist()
    D = D[0].tolist()
    results = []
    for idx, score in zip(I, D):
        meta = chunks_meta[idx]
        results.append({"text": meta["text"], "doc_id": meta["doc_id"], "score": float(score)})
    return results

In [8]:
from transformers import AutoTokenizer as HFTokenizer, AutoModelForCausalLM, pipeline

# 本地 Qwen LLM
qwen_tok = HFTokenizer.from_pretrained(QWEN_DIR, local_files_only=True, trust_remote_code=True)
qwen_model = AutoModelForCausalLM.from_pretrained(QWEN_DIR, local_files_only=True, trust_remote_code=True).to(DEVICE).eval()

# transformers 的 pipeline（别和上面的“自定义分词器”混用，这是给 LLM 生成用的）
qwen_pipe = pipeline(
    "text-generation",
    model=qwen_model,
    tokenizer=qwen_tok,
    device=0 if DEVICE == "cuda" else -1,
    max_new_tokens=256,
    do_sample=False,
)

In [9]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 把 transformers 的 pipeline 包装成 LangChain 的 LLM
llm_langchain = HuggingFacePipeline(pipeline=qwen_pipe)

# Prompt 模板
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "你是一名医疗健康助手。请根据以下资料回答问题：\n"
        "{context}\n\n"
        "问题：{question}\n回答："
    ),
)

# LLMChain
rag_chain = LLMChain(prompt=prompt_template, llm=llm_langchain)

# 生成函数（替换你原来的 generate_answer）
# === 替换 generate_answer 函数（兼容 ctx 为字符串或字典）===
def generate_answer(query: str, top_k=TOP_K):
    ctxs = retrieve(query, top_k=top_k)           # ctxs: 可能是 dict 或 str

    # 统一拿到字符串
    ctx_texts = [c["text"] if isinstance(c, dict) else c for c in ctxs]

    # 拼接给 LLM 的上下文
    context_text = "\n".join([f"- {t}" for t in ctx_texts])

    # LangChain 调用
    raw_output = rag_chain.run({"context": context_text, "question": query})

    # 轻度清理
    answer = raw_output.split("回答：")[-1].strip()
    answer = answer.replace("你是一名医疗健康助手。", "").strip()

    # 返回答案 + 纯文本 ctx 列表
    return answer, ctx_texts

/tmp/ipykernel_908/1653740109.py:6: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  llm_langchain = HuggingFacePipeline(pipeline=qwen_pipe)
/tmp/ipykernel_908/1653740109.py:19: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  rag_chain = LLMChain(prompt=prompt_template, llm=llm_langchain)


In [10]:
# 测试：单条
ans, ctxs = generate_answer("糖尿病人早餐可以吃什么？")
print("答案：", ans)
print("检索到的文档：", [c[:50] for c in ctxs])  # 这里改成直接切片字符串

# 批量测试
test_queries = [
    "糖尿病患者早餐可以吃什么？",
    "胃炎反复发作应该怎么调理？",
    "孕期贫血需要补充哪些营养？"
]

for q in test_queries:
    ans, ctxs = generate_answer(q)
    print(f"\n问题: {q}")
    print("答案:", ans)
    print("Top-3 检索片段:", [c[:50] for c in ctxs[:3]])

Encoding: 100%|██████████| 1/1 [00:00<00:00,  2.41it/s]
/tmp/ipykernel_908/1653740109.py:33: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  raw_output = rag_chain.run({"context": context_text, "question": query})


答案： 根据你的描述属于
检索到的文档： ['你好，孩子缺钙,要补钙和鱼肝油', '你好，有糖尿病的话，饮食上需要注意不能吃含糖食物，注意控制能量摄入水平。可以常吃蔬菜或低糖含量的水果', '你好属于慢性感染引起的牙龈炎，需要停止哺乳，用吸奶器吸奶，因对因而有影响，增加营养补充维生素微量元素', '根据你的描述属于正常情况有关，一般是可以不用治疗的，需要正确对待增加营养补充维生素微量元素，易消化易', '根据你的描述症状有可能避孕失败，需要上医院复查，建议增加营养补充维生素补充蛋白质，易消化易吸收饮食，']


Encoding: 100%|██████████| 1/1 [00:00<00:00, 76.95it/s]



问题: 糖尿病患者早餐可以吃什么？
答案: 糖尿病患者早餐可以吃一些低糖、高纤维的食物，如燕麦粥、全麦面包、鸡蛋、牛奶、水果等。同时，建议糖尿病患者在饮食上要控制总热量，避免过量摄入糖分和脂肪，以维持血糖水平的稳定。
Top-3 检索片段: ['你好，孩子缺钙,要补钙和鱼肝油', '你好属于慢性感染引起的牙龈炎，需要停止哺乳，用吸奶器吸奶，因对因而有影响，增加营养补充维生素微量元素', '尿路感染引起的症状建议及时的多喝水注意休息均衡营养多吃新鲜蔬菜水果口服抗生素和输液抗生素的方法治疗口']


Encoding: 100%|██████████| 1/1 [00:00<00:00, 75.13it/s]



问题: 胃炎反复发作应该怎么调理？
答案: 胃炎反复发作，建议您采取以下措施进行调理：

1. **饮食调整**：避免辛辣、油腻、过热或过冷的食物，减少咖啡因和酒精的摄入。选择易消化、营养丰富的食物，如粥、面条、蒸蛋等。

2. **规律作息**：保持充足的睡眠，避免熬夜，有助于身体恢复和免疫系统的正常运作。

3. **适量运动**：适当的体育活动可以增强体质，改善消化功能，但应避免剧烈运动。

4. **心理调适**：保持良好的心态，避免过度紧张和焦虑，因为情绪波动也可能影响胃部健康。

5. **药物治疗**：根据医生的指导使用抗酸药、胃黏膜保护剂等药物，以减轻症状和促进愈合。

6. **定期复查**：定期到医院进行胃镜检查和其他相关检查，以便及时了解病情变化并调整治疗方案。

7. **中医调理**：可以考虑采用中药调理，如服用具有健脾和胃作用的中药方剂，但需在专业中医师的指导下进行。

请注意，以上建议仅供参考，具体治疗方案应由专业医生根据您的具体情况制定。如果症状持续或加重，请及时就医。
Top-3 检索片段: ['请问我女儿两岁半了，现在发烧38.2度，可是不吃药，怎么办？38,2度是不是烧的很严重？怎么治疗？', '你好，孩子缺钙,要补钙和鱼肝油', '尿路感染引起的症状建议及时的多喝水注意休息均衡营养多吃新鲜蔬菜水果口服抗生素和输液抗生素的方法治疗口']


Encoding: 100%|██████████| 1/1 [00:00<00:00, 62.50it/s]



问题: 孕期贫血需要补充哪些营养？
答案: 孕期贫血需要补充的营养包括：正确对待增加营养补充维生素微量元素，易消化易吸收饮食。
Top-3 检索片段: ['你好，孩子缺钙,要补钙和鱼肝油', '你好属于慢性感染引起的牙龈炎，需要停止哺乳，用吸奶器吸奶，因对因而有影响，增加营养补充维生素微量元素', '你好，钙片奶粉可以同时吃的，注意孕期保健和日常护理，适当补充微量元素为好的，定期孕检为好的，注意健康']


In [11]:
!jupyter nbconvert --to html RAG_outputTest.ipynb

[NbConvertApp] Converting notebook RAG_outputTest.ipynb to html
[NbConvertApp] Writing 630095 bytes to RAG_outputTest.html
